In [9]:
from trl import apply_chat_template
from transformers import AutoTokenizer
from datasets import load_dataset
import datasets
train_dataset = datasets.load_from_disk("countdown_tasks")
test_dataset = datasets.load_from_disk("countdown_tasks")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

In [10]:
train_dataset = train_dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})

Map: 100%|██████████| 49037/49037 [00:01<00:00, 25669.48 examples/s]


In [ ]:
test_dataset = test_dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})

In [2]:
from trl import GRPOConfig, GRPOTrainer
import re

def format_reward_func(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>.*?</think><answer>.*?</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]

def reward_func(completions, ground_truth, **kwargs):
    # Regular expression to capture content inside <answer> tags
    matches = [re.search(r"<answer>(.*?)</answer>", completion) for completion in completions]
    contents = [match.group(1) if match else "" for match in matches]
    # Reward 1 if the content is the same as the ground truth, 0 otherwise
    rewards = []
    for c, gt in zip(contents, ground_truth):
        try:
            calculation_result = eval(c)
            if calculation_result == gt:
                rewards.append(1.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

training_args = GRPOConfig(output_dir="Qwen2-0.5B-GRPO", logging_steps=10)
trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",
    reward_funcs=[format_reward_func, reward_func],
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

/Users/shinnosukeuesaka/continuous-cot-rl/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 